In [1]:
import cv2
import os
import re 
import warnings
import dlib
import numpy as np
import glob
import matplotlib.pyplot as plot

from sklearn.datasets import fetch_olivetti_faces
faces = fetch_olivetti_faces()


# Dataset Implementation

In [2]:
base_directory = 'Dataset/VISA_Face/VISA_Face'
face_images = []

def parse_face_dataset():
    for path in glob.iglob(base_directory + '/*'):
        filename = os.path.basename(path)
        
        # string manipulation
        underscore_index = filename.find("_")
        filename_parsed = filename[:underscore_index]    
        match = re.search(r"(.*?)_2017_001", filename)
        if match:
            filename_parsed = match.group(1)
        else:
            warnings.warn(f"No match found for filename: {filename}")
            continue  # Skip processing this file
        
        label = filename_parsed
        image_id = 0
        
        for image_path in glob.iglob(path + '/*'):
            try:
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if image is None:
                    warnings.warn(f"Failed to load image: {image_path}")
                    continue
                image = cv2.resize(image, (400, 300))  # Resize image to reduce memory usage
                face_images.append([image, image_id, label])
                image_id += 1
            except Exception as e:
                warnings.warn(f"Error processing image: {image_path}\n{e}")
            
    print('Total Face Images Found: ' + str(len(face_images)))   
    return face_images


# Face Detection Function

In [3]:

def face_detection(face_images, display):
    pre_processed_images = []
    print("Face Image Preprocessing STARTED...")
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
    
    # Create output directory if it doesn't exist
    output_dir = os.path.join('Face_Output', 'Face_Output_Detection')
    os.makedirs(output_dir, exist_ok=True)

    # Clear output directory if it already exists
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}: {e}")

    for face_image in face_images:
        (image, image_id, label) = face_image
        image_id += 1
        faces = face_cascade.detectMultiScale(image, 1.1, 4)

        for (x, y, width, height) in faces:
            face = image[y:y + height, x:x + width]

            # Save cropped face image
            output_path = os.path.join(output_dir, f'{label}_{image_id}_Cropped.jpg')
            cv2.imwrite(output_path, face)

            pre_processed_images.append([face, image_id, label])

    print("Face Image Preprocessing COMPLETE...")

# Feature Extraction Function

In [4]:
def facial_feature_extraction(output_dir):
    # Initialize face detector and shape predictor
    print("Face Feature Extraction STARTED...")
    detector = dlib.get_frontal_face_detector()
    predictor_path = 'Dependencies/shape_predictor_68_face_landmarks.dat'
    predictor = dlib.shape_predictor(predictor_path)
    
    # Create output directory if it doesn't exist
    output_dir = os.path.join(output_dir, 'Face_Output_Feature_Extraction')
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to the directory containing cropped face images
    detection_dir = os.path.join('Face_Output', 'Face_Output_Detection')

    # Clear output directory if it already exists
    for filename in os.listdir(output_dir):
        file_path = os.path.join(output_dir, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}: {e}")
    
    # Iterate over images in the detection directory
    for filename in os.listdir(detection_dir):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            # Read the image
            image_path = os.path.join(detection_dir, filename)
            image = cv2.imread(image_path)
            if image is None:
                continue

            # Detect faces in the image
            dets = detector(image, 1)
            
            # Iterate over detected faces
            for i, d in enumerate(dets):
                # Predict facial landmarks
                shape = predictor(image, d)
                
                # Draw landmarks on the image
                for i in range(68):
                    cv2.circle(image, (shape.part(i).x, shape.part(i).y), 1, (0, 255, 0), -1)
                    
                # Draw rectangle around the face
                cv2.rectangle(image, (d.left(), d.top()), (d.right(), d.bottom()), (255, 0, 0), 2)
                    
            # Save image with landmarks and detected faces
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)
    
    # Print message when feature extraction is done
    print("Face Feature Extraction COMPLETE...")

# Example usage
output_directory = 'Face_Output'


# Main Function

In [5]:
if __name__ == "__main__":
    face_images = parse_face_dataset()
    face_detection(face_images, display = False)  # Suppress display for face detection
    facial_feature_extraction(output_directory)

C:\Users\saish\AppData\Local\Temp\ipykernel_16248\3743092712.py:15: UserWarning: No match found for filename: olivetti_py3.pkz
  warnings.warn(f"No match found for filename: {filename}")


Total Face Images Found: 558
Face Image Preprocessing STARTED...
Face Image Preprocessing COMPLETE...
Face Feature Extraction STARTED...
Face Feature Extraction COMPLETE...
